# Delaunay-Watershed for geometry reconstruction: cell aggregate example

In this notebook:
- We show how to construct a multimaterial mesh from segmentation masks of cells in an aggregate.
- We show how to easily visualize the mesh.

# 0) Install Delaunay-Watershed

We install the viewing option from Delaunay-Watershed, based on Napari and Polyscope, to visualize the mesh later.

In [ ]:
pip install "dw3d[viewing]"

## 1) Compute multimaterial mesh with Delaunay Watershed

In [ ]:
import skimage.io as io
from dw3d import GeometryReconstruction3D

dist = 3

image_path = "../data/Images/1.tif"
image = io.imread(image_path)
reconstruction = GeometryReconstruction3D(image, min_dist = dist, print_info=True)

# The multimaterial mesh can be saved in a .vtk or .rec format. In binary or ASCII.
reconstruction.save_to_rec_mesh("actual_1.rec", binary_mode=False)
reconstruction.save_to_vtk_mesh("actual_1.vtk", binary_mode=True)
print("Mesh saved.")

In [ ]:
# The multimaterial mesh is:
# points: an array of 3D points (geometry)
# triangles: an array of triangles (topology)
# labels: an array of materials on each side of a triangle.
points, triangles, labels = reconstruction.mesh

print(f"{points=}")
print(f"{triangles=}")
print(f"{labels=}")


## 2) Visualize mesh with our Polyscope viewer.

In [4]:
from dw3d.viewing import plot_cells_polyscope

plot_cells_polyscope(reconstruction)

# sometimes, it's better to scatter cells to understand the geometry
# plot_cells_polyscope(reconstruction, view="Scattered", scattering_coeff=0.5)

# 3) Understand the construction process with Napari

In [ ]:
from dw3d.viewing import plot_in_napari

v = plot_in_napari(reconstruction, add_mesh=True)